<a href="https://colab.research.google.com/github/kamantina/projectreport/blob/main/feature_analysis/comments_language_checking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# Load the CSV file (change the path to your file path)
df = pd.read_csv('/content/drive/MyDrive/CISC7298/comments_202050308_utf8.csv', encoding='utf-8')

# Ensure necessary columns exist
if 'videoId' not in df.columns or 'textOriginal' not in df.columns:
    raise ValueError("The required columns 'videoId' and 'textOriginal' are missing in your CSV.")

# Handle missing values
df['textOriginal'] = df['textOriginal'].fillna('').astype(str)

print("Data loaded successfully.")

Data loaded successfully.


Keep only top comments

In [ ]:
df = df[df['isTopComment'] == 1]

Remove URLs, hashtags and special characters but keep letters, numbers, spaces and emojis

In [ ]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 7.6 MB/s eta 0:00:00


In [ ]:
import re
import emoji

def clean_comment(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)
    # Remove hashtags and symbols
    text = re.sub(r'[#@]', '', text)
    # Remove special characters except letters, numbers, spaces, and emojis
    text = re.sub(r'[^\w\s@#$%^&*()_+={}\[\]:;,.!?~`\-\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U000024C2-\U0001F251]', '', text)
    return text.strip()

Use langid to detect language

In [ ]:
!pip install langid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 17.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941171 sha256=9e97ae69b9332c741f3fac05d28e755d0ff047fa1463bf21e54bd7bd61e5e582
  Stored in directory: /root/.cache/pip/wheels/32/6a/b6/b7eb43a6ad55b139c15c5daa29f3707659cfa6944d3c696f5b
Successfully built langid


In [ ]:
import langid

def is_english(text, threshold=0.1):
    """
    Uses langid for language detection.
    Returns True if the detected language is English.
    """
    try:
        # First, convert emojis to descriptive words.
        # preprocessed_text = preprocess_text(text)
        lang, confidence = langid.classify(text)
        return lang == 'en' and confidence >= threshold
    except Exception:
        return False


Manually check first 200 rows

In [ ]:
# Sample the first 200 rows
df_sample = df.head(200).copy()
df_sample["cleaned_text"] = df_sample["textOriginal"].apply(preprocess_text)
df_sample["is_english"] = df_sample["cleaned_text"].apply(is_english)

In [ ]:
df_sample

Go ahead to apply English checking to all comments once settings are ready

In [ ]:
# Load your comments into a DataFrame (df_comments)
df["cleaned_text"] = df["textOriginal"].apply(preprocess_text)
df["is_english"] = df["cleaned_text"].apply(is_english)

Check the language again on both the original text and cleaned text of comments labelled as "False" for English

In [ ]:
# Filter rows where is_english is False
false_rows = df[df['is_english'] == False].copy()

# Apply langid to detect the language of the "textOriginal" column
false_rows['detected_language'] = false_rows['textOriginal'].apply(lambda text: langid.classify(text)[0])
false_rows['cleaned_detected_language'] = false_rows['cleaned_text'].apply(lambda text: langid.classify(text)[0])


In [ ]:
false_rows['detected_language'].value_counts()

,count
detected_language,
es,6574
ru,6354
en,6057
de,5007
ko,4160
...,...
or,19
zu,18
te,15


In [ ]:
false_rows['cleaned_detected_language'].value_counts()

,count
cleaned_detected_language,
es,7694
ru,6358
de,5963
it,4445
fr,4208
...,...
pa,2
gu,2
or,2


In [ ]:
false_english_rows = false_rows[false_rows['detected_language'] == 'en']

In [ ]:
false_english_rows.shape

(6057, 11)

Update all comments marked as "en" for original and cleaned text as "True" for English checking

In [ ]:
# Update the isEnglish column in false_rows
false_rows.loc[
    (false_rows['detected_language'] == 'en') | (false_rows['cleaned_detected_language'] == 'en'),
    'isEnglish'
] = True  # Change to True for rows where either detect_language or cleaned_detect_language is "en"

In [ ]:
false_rows['isEnglish'].value_counts()

,count
isEnglish,
True,6064


In [ ]:
false_rows.shape

(62970, 12)

In [ ]:
# Step 1: Create a new column 'isEnglish2' by copying the values from 'isEnglish'
df['is_english2'] = df['is_english']

# Step 2: Get the list of CommentIds in false_rows where isEnglish=True
comment_ids_to_update = false_rows[false_rows['isEnglish'] == True]['commentId']

# Step 3: Update isEnglish2 in the original df for matching CommentIds
df.loc[df['commentId'].isin(comment_ids_to_update), 'is_english2'] = True


In [ ]:
df['is_english'].value_counts()

,count
is_english,
True,459318
False,62970


In [ ]:
df['is_english2'].value_counts()

,count
is_english2,
True,465382
False,56906


In [ ]:
df.to_csv('comments_202050308_utf8_isEnglish2.csv', index=False)

Create a pivot table to count the number Engish and non-English comments in each video as well as the percentage of non-English comments

In [ ]:
# Create a pivot table where rows are videoId and columns are is_english, filled with counts
pivot_df = df.pivot_table(index="videoId", columns="is_english2", aggfunc="size", fill_value=0)

# If pivot_df columns are booleans, then:
# pivot_df[False] counts non-English comments, and pivot_df[True] counts English comments.
pivot_df["total"] = pivot_df[False] + pivot_df[True]
pivot_df["percentage_false"] = pivot_df[False] / pivot_df["total"] * 100

# If you want to reset index for easier reading, you can:
pivot_df = pivot_df.reset_index()
print(pivot_df)

# Filter out videos and comments with threshold on minimum number of comments and pecentage of English comments




In [ ]:
# Load the video data obtained from YouTube Data API
df_video = pd.read_csv('/content/drive/MyDrive/CISC7298/videos_202500308_utf8_filtered.csv', encoding='utf-8')

print("Data loaded successfully.")

Data loaded successfully.


In [ ]:
# Merge the original video data file and the pivot table with counts on English and non-English comment on the 'videoId' column
df_video = df_video.merge(pivot_df, on='videoId', how='left')

# Verify the merged DataFrame
print(df_video.head())

In [ ]:
# Filter rows based on the conditions
filtered_df = df_video.loc[(df_video['True'] >= 20) & (df_video['percentage_false'] < 50)]

# Display the filtered rows
print(filtered_df)

In [ ]:
filtered_df['True'].sum()

np.float64(461534.0)

In [ ]:
# Filter rows where isEnglish2 == True
df_comment_filtered = df_comment[df_comment['is_english2'] == True]

In [ ]:
df_comment_filtered.shape

(465382, 10)

In [ ]:
# Filter rows in df_comment_filtered based on videoId in filtered_df
df_comment_filtered = df_comment_filtered[df_comment_filtered['videoId'].isin(filtered_df['videoId'])]

In [ ]:
df_comment_filtered.shape

(461534, 10)

Save the filtered comment data and video data files for further process.

In [ ]:
# Save the comment data file (change the file name)
df_comment_filtered.to_csv("comments_202050308_utf8_isEnglish2_2050.csv", index=False)

In [ ]:
# Save the video data (change the file name)
filtered_df.to_csv("videos_202500308_utf8_filtered_isEnglish2_2050.csv", index=False)